In [ ]:
!pip install openbox
!pip install shapely
!pip install plotly
!pip install torch

In [ ]:
class Data:
    def __init__(self, sensorPositions, space, epsilon):
        self.radius = 1
        self.placeHolders = sensorPositions
        self.epsilon = epsilon
        self.space = space
        # self.SensorPlaceHolderSetup()
        
    def frange(self, start, stop, step):
        steps = []
        while start <= stop:
            steps.append(start)
            start +=step
            
        return steps

    # def SensorPlaceHolderSetup(self):
    #     Xs = self.frange(0, self.space[0], self.epsilon)
    #     Ys = self.frange(0, self.space[1], self.epsilon)
            
    #     for x in Xs:
    #       for y in Ys:
    #         self.placeHolders.append([x, y])

    def GetSensorConfiguration(self):
        from collections import Counter
        sensorLocations, sensorTypes = self.GetSensorLocations()
        
        _, rooms = pf.ParseWorld(simworldname = '')
        
        summaryDict = Counter(sensorTypes)

        # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
        configurationSummary = []
        for key in summaryDict:
            if (key == 1):
                configurationSummary.append(['motion sensors', summaryDict[key]])

            elif (key == 2):
                configurationSummary.append(['beacon sensors', summaryDict[key]])

        configurationDetails = []
        for index, loc in enumerate(sensorLocations):
            room = ""
            for r in rooms:
                if loc[0] >= rooms[r][0][0] and loc[0] <= rooms[r][1][0] and loc[1] >= rooms[r][0][1] and loc[1] <= rooms[r][1][1]:
                    room = r
                    break

            if (sensorTypes[index] == 1):
                configurationDetails.append(tuple([loc, room, 'motion sensors']))

            elif (sensorTypes[index] == 2):
                configurationDetails.append(tuple([loc, room, 'beacon sensors']))

        return [[configurationSummary, [tuple(configurationDetails)]], self.radius]


    def GetSensorLocations(self):
        sensorLocations = []
        sensorTypes = []
        for index, sensorIndicator in enumerate(self.placeHolders):
            sensorLocations.append(self.placeHolders[index])

            # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
            sensorTypes.append(1)


        return sensorLocations, sensorTypes


class BOVariables:
    def __init__(self, Data_path, epsilon, initSensorNum, maxSensorNum, radius, sampleSize):
        self.epsilon = epsilon
        self.Data_path = Data_path
        self.initSensorNum = initSensorNum
        self.maxSensorNum = maxSensorNum
        self.radius = radius
        self.sensor_distribution, self.types, self.space, self.rooms, self.agentTraces = self.ModelsInitializations()

    def ModelsInitializations(self):
        #----- Space and agent models -----: 
        # simworldname = self.Data_path + '/Configuration Files/simulationWorld2.xml'
        simworldname = ''
        agentTraces = []
        directory = os.fsencode(self.Data_path + 'Agent Trace Files/')
            
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".csv"): 
                agentTraces.append(self.Data_path + 'Agent Trace Files/' + filename)

        # Parsing the space model: 
        space, rooms = pf.ParseWorld(simworldname)

        xs = []
        for i in space:
          for j in i:
            xs.append(j)
        A = list(set(xs))
        A.sort()
        space = [A[-1], A[-2]]

        # User parameters 
        types, sensor_distribution = pf.GetUsersParameters()

        roomsList = []
        for room in sensor_distribution:
            roomsList.append(room)
              
        return sensor_distribution, types, space, rooms, agentTraces


def frange(start, stop, step):
    steps = []
    while start <= stop:
        steps.append(start)
        start +=step
        
    return steps

def MakeSensorCombinations(start, end, epsilon, sensorType, room):
    a1, b1 = makeBoundaries(epsilon, start[0], end[0])
    a2, b2 = makeBoundaries(epsilon, start[1], end[1])    
    Xs = frange(a1, b1, epsilon)
    Ys = frange(a2, b2, epsilon)
    
    points = list(itertools.product(list(itertools.product(Xs, Ys)), [room], [sensorType[0]])) 
    C = itertools.combinations(points, distribution[room][types.index(sensorType)])

    return C

def PreProcessor(df):
    # df['motion sensors'] = df['motion sensors'].apply(ast.literal_eval)
    df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
    #df['motion sensors'] = df['motion sensors'].apply(lambda s: s + [1])
    # df['beacon sensors'] = df['beacon sensors'].apply(ast.literal_eval)
    try:
      df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
    except:
      pass

    pre_activity = ''
    save_index = 0

    for index, row in df.iterrows():
        save_index = index
        Activity = row['activity']

        if Activity != pre_activity:
            if pre_activity != '':
                df.at[index - 1, 'motion sensors'] += [0]
            else:
                df.at[index, 'motion sensors'] += [1]

            pre_activity = Activity
        else:
            df.at[index - 1, 'motion sensors'] += [1]

    df.at[save_index, 'motion sensors'] += [0]

    sensors = set([])

    previous_M = None
    previous_B = None
    output_file = []

    for index, row in df.iterrows():
      T = row['time']
      M = row['motion sensors']
      try:
        B = row['beacon sensors']
      except:
        pass

      Activity = row['activity']
      Activity = Activity.replace(' ', '_')
      MotionSensor_Names = []
      sensorNames = []
      MotionSensor_Message = []
      BeaconSensor_Names = []
      BeaconSensor_Message = []
      

      # time = convertTime(T)
      time = "2020-06-16 " + T + ".00"

      # Motion Sensor
      for i in range(len(M)):
        sensorNames.append(Name(i, 'M'))
        if M[i] == 1:
          if (previous_M != None):
            if (previous_M[i] == 0):
              MotionSensor_Names.append(Name(i,'M'))
              MotionSensor_Message.append('ON')

          else:
            MotionSensor_Names.append(Name(i,'M'))
            MotionSensor_Message.append('ON')

        if previous_M != None:
          if M[i] == 0 and previous_M[i] == 1:
            MotionSensor_Names.append(Name(i,'M'))
            MotionSensor_Message.append('OFF')

      previous_M = M
      # Beacon Sensor

      try:
        for i in range(len(B)):
          sensorNames.append(Name(i, 'B'))
          if B[i] == 1:
            BeaconSensor_Names.append(Name(i,'B'))
            BeaconSensor_Message.append('ON')
          if previous_B != None:
            if B[i] == 0 and previous_B[i] == 1: 
              BeaconSensor_Names.append(Name(i,'B'))
              BeaconSensor_Message.append('OFF')
        previous_B = B

      except:
        pass

      for m in range(len(MotionSensor_Names)):
        output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
        
      for s in sensorNames:
          sensors.add(s)

    return output_file, list(sensors)

#returns the name of the sensor
def Name(number, typeSensor):
    if number < 10:
      return typeSensor + str(0) + str(number)
    else:
      return typeSensor + str(number)

#converts epoch time to human readable
def convertTime(posix_timestamp):
    tz = pytz.timezone('MST')
    dt = datetime.fromtimestamp(posix_timestamp, tz)
    time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return time

def MakeDataBoundaries(height = 10.5, width = 6.6, MaxSensors = 15):
    from collections import defaultdict, OrderedDict

    d = dict()

    for idx in range(MaxSensors):
            d['x' + str(idx)] = (0.5, width - 0.5)
            d['y' + str(idx)] = (0.5, height - 0.5)

    return d

def black_box_function(sample, simulateMotionSensors = True, simulateEstimotes = False, Plotting = False):       
    files = []

    import sys
    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/SensorDeploymentOptimization/')
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/SensorDeploymentOptimization/'

    else:
        # sys.path.append('../../Codes/SensorDeploymentOptimization/')
        sys.path.append('..')
        Data_path = '../../Codes/SensorDeploymentOptimization/'

    all_sensors = set([])

    for agentTrace in BOV.agentTraces:
        df_ = sim_sis.RunSimulator(BOV.space, 
                                   BOV.rooms, 
                                   agentTrace,
                                   sample.GetSensorConfiguration(), 
                                   simulateMotionSensors, 
                                   simulateEstimotes, 
                                   Plotting, 
                                   BOV.Data_path)
        
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        files.append(dataFile)
        
    
    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')

    else:
        sys.path.append('../../Codes/CASAS/AL-Smarthome')

    import al
    import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    f1_score = (al.leave_one_out(files, all_sensors)[0]) * 100

    if f1_score < 0:
        f1_score = 0

    return 100 - f1_score

def function_to_be_optimized(config):
    sensorPositions = []
    sensor_xy = []

    for i in range(1, CONSTANTS['max_sensors'] + 1):
        if (config['x' + str(i)] > 0):
            sensor_xy.append(config['x' + str(i)])
            sensor_xy.append(config['y' + str(i)])
            sensorPositions.append(sensor_xy)
            sensor_xy = []
        
          
    # print(sensorPositions)
    data = Data(sensorPositions, BOV.space, CONSTANTS['epsilon'])
    
    # print(data.GetSensorConfiguration())
    # result = dict()

    # result['objs'] = np.stack([black_box_function(data), len(sensorPositions)], axis=-1)
    
    return black_box_function(data)

In [ ]:
CONSTANTS = {
    'iterations': 100,
    'initial_samples': 10,
    'restarts_number': 10,
    'raw_samples': 10,
    'candidates_num': 1,
    'sequential': True,
    'epsilon': 1,
    'radius': 1,
    'height': 8.0,
    'width': 8.0,
    'max_sensors': 15
}

runningOnGoogleColab = True
    
if __name__ ==  '__main__':
    import sys

    # if (runningOnGoogleColab == True):
    #     from google.colab import drive    
    #     drive.mount('/content/gdrive', force_remount=True)
    #     Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
    #     sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        
    # else:
    #     Data_path = '../../Codes/Sensor Simulator/'
    #     sys.path.append('../../Codes/Sensor Simulator/')

    if (runningOnGoogleColab == True):
        from google.colab import drive    
        drive.mount('/content/gdrive', force_remount=True)
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/SensorDeploymentOptimization/'
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/SensorDeploymentOptimization/')

    else:
        Data_path = '../../Codes/SensorDeploymentOptimization/'
        # sys.path.append('../../Codes/SensorDeploymentOptimization/')
        sys.path.append('..')
    
    # from ax import optimize, ChoiceParameter
    from scipy.stats import norm
    from numpy import argmax
    import SIM_SIS_Libraries.SensorsClass
    import SIM_SIS_Libraries.SIM_SIS_Simulator as sim_sis
    import SIM_SIS_Libraries.ParseFunctions as pf
    import itertools
    import numpy as np
    import pandas as pd
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde
    import copy
    from datetime import datetime
    import pytz
    import ast
    import os
    import random
    import plotly
    import torch

    finalResults = []
    w = CONSTANTS['width'] - 0.5
    h = CONSTANTS['height'] - 0.5

    dataBoundaries = MakeDataBoundaries(
                                        height = CONSTANTS['height'], 
                                        width = CONSTANTS['width'], 
                                        MaxSensors = CONSTANTS['max_sensors']
                                       )

    BOV =  BOVariables(
                       Data_path, 
                       CONSTANTS['epsilon'], 
                       CONSTANTS['max_sensors'], 
                       CONSTANTS['max_sensors'], 
                       CONSTANTS['radius'],
                       CONSTANTS['initial_samples']
                      )

    from openbox import sp
    

    # Define Search Space
    space = sp.Space()

    list_of_variables = []
    for i in range(1, CONSTANTS['max_sensors'] + 1):
        x = sp.Int("x" + str(i), 0, CONSTANTS['width'] - 1, default_value=7)
        y = sp.Int("y" + str(i), 1, CONSTANTS['height'] - 1, default_value=2)
        list_of_variables.append(x)
        list_of_variables.append(y)
    
    space.add_variables(list_of_variables)

    from openbox import Optimizer

    history_list = []
    for index in range(10):
        opt = Optimizer(
            function_to_be_optimized,
            space,
            # num_objs=2,
            max_runs=500,
            surrogate_type='prf',
            # acq_type = 'ehvi',
            acq_optimizer_type='random_scipy',
            time_limit_per_trial=31000,
            task_id='Shadan',
            # init_strategy='sobol',
            # ref_point=[100, 15]
        )
        history = opt.run()
        history.save_json(fn = '.\openbox_results\history' + str(index) + '.json')
        history_list.append(history)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot pareto front
pareto_front = np.asarray(opt.get_history().get_pareto_front())
if pareto_front.shape[-1] in (2, 3):
    if pareto_front.shape[-1] == 2:
        plt.scatter(pareto_front[:, 0], pareto_front[:, 1])
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
    elif pareto_front.shape[-1] == 3:
        ax = plt.axes(projection='3d')
        ax.scatter3D(pareto_front[:, 0], pareto_front[:, 1], pareto_front[:, 2])
        ax.set_xlabel('Objective 1')
        ax.set_ylabel('Objective 2')
        ax.set_zlabel('Objective 3')
    plt.title('Pareto Front')
    plt.show()

In [ ]:
# plot hypervolume
data = opt.get_history().data

sensors_num = []
accuracies = []

print(data)

for _ in range(len(data)):
    d = data.popitem(last=True)
    print(d)
    accuracies.append(d[0])
    sensors_num.append(d[1])

print(accuracies)
    
plt.plot(accuracies)
    
# for item in hypervolume:
    # print(item.get_array())
    # print(item)

# log_hv_diff = 100 - np.asarray(hypervolume)
# plt.plot(log_hv_diff)
# plt.xlabel('Iteration')
# plt.ylabel('Log Hypervolume Difference')
# plt.show()

In [ ]:
opt.get_history()

In [ ]:
for index, history in enumerate(history_list):
    history.save_json(fn = '.\openbox_results\history' + str(index) + '.json')

In [ ]:
def my_plot_convergence(
            results,
            xlabel="Number of iterations $n$",
            ylabel=r"Min objective value after $n$ iterations",
            ax=None, name=None, alpha=0.2, yscale=None,
            color=None, true_minimum=None,
            **kwargs):
        """Plot one or several convergence traces.
        Parameters
        ----------
        args[i] :  `OptimizeResult`, list of `OptimizeResult`, or tuple
            The result(s) for which to plot the convergence trace.
            - if `OptimizeResult`, then draw the corresponding single trace;
            - if list of `OptimizeResult`, then draw the corresponding convergence
              traces in transparency, along with the average convergence trace;
            - if tuple, then `args[i][0]` should be a string label and `args[i][1]`
              an `OptimizeResult` or a list of `OptimizeResult`.
        ax : `Axes`, optional
            The matplotlib axes on which to draw the plot, or `None` to create
            a new one.
        true_minimum : float, optional
            The true minimum value of the function, if known.
        yscale : None or string, optional
            The scale for the y-axis.
        Returns
        -------
        ax : `Axes`
            The matplotlib axes.
        """
        losses = list(results)

        n_calls = len(losses)
        iterations = range(1, n_calls + 1)
        maxs = [np.max(losses[:i]) for i in iterations]
        min_maxs = min(maxs)
        cliped_losses = np.clip(losses, min_maxs, None)
        return plot_convergence(iterations, maxs, cliped_losses, xlabel, ylabel, ax, name, alpha, yscale, color,
                                true_minimum, **kwargs)
    
def plot_convergence(
        x, y1, y2,
        xlabel="Number of iterations $n$",
        ylabel=r"Min objective value after $n$ iterations",
        ax=None, name=None, alpha=0.2, yscale=None,
        color=None, true_minimum=None,
        **kwargs):
    """Plot one or several convergence traces.
    Parameters
    ----------
    args[i] :  `OptimizeResult`, list of `OptimizeResult`, or tuple
        The result(s) for which to plot the convergence trace.
        - if `OptimizeResult`, then draw the corresponding single trace;
        - if list of `OptimizeResult`, then draw the corresponding convergence
          traces in transparency, along with the average convergence trace;
        - if tuple, then `args[i][0]` should be a string label and `args[i][1]`
          an `OptimizeResult` or a list of `OptimizeResult`.
    ax : `Axes`, optional
        The matplotlib axes on which to draw the plot, or `None` to create
        a new one.
    true_minimum : float, optional
        The true minimum value of the function, if known.
    yscale : None or string, optional
        The scale for the y-axis.
    Returns
    -------
    ax : `Axes`
        The matplotlib axes.
    """
    if ax is None:
        ax = plt.gca()

    ax.set_title("Convergence plot")
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.grid()

    if yscale is not None:
        ax.set_yscale(yscale)

    ax.plot(x, y1, c=color, label=name, **kwargs)
    ax.scatter(x, y2, c=color, alpha=alpha)

    if true_minimum is not None:
        ax.axhline(true_minimum, linestyle="--",
                   color="r", lw=1,
                   label="True minimum")

    if true_minimum is not None or name is not None:
        ax.legend(loc="upper right")
    return ax

In [ ]:
import os
from openbox.utils import history_container

history_list = []


for filename in os.listdir('.\openbox_results'):
    history = history_container.HistoryContainer(task_id = 'SPO')
    hist = history.load_history_from_json(cs = space, fn = '.\openbox_results\\' + filename)
    history_list.append(hist)

In [ ]:
for c in history_list[0].items():
    print(c[1])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

for i in range(len(history_list)):
    # print(hist)
    # for c in history_list[0].items():
    # print(c[1])
    # hist.plot_convergence()
    # l = [100 - p for p in hist.perfs]
    # figure(figsize=(8, 6), dpi=80)
    # print(max(l))
    # print(min(l))
    # data = history_list[i].items()
    # print(data[1])
    # my_plot_convergence(results = data) 
    # plt.show()
    data = []
    for c in history_list[i].items():
        data.append(100 - c[1])
    
    # print(data)
    my_plot_convergence(results = data)
    plt.show()


In [ ]:
history_list

In [ ]:
    
for index, history in enumerate(history_list):
    # print(history.perfs)
    figure(figsize=(8, 6), dpi=80)
    history.plot_convergence()
    print(history.min_y)
    plt.show()

In [ ]:
rooms = {'bedroom': [(3.9, 0.0), (8.0, 4.1)],
         'livingroom': [(0.0, 1.9), (6.3, 4.3)],
         'diningroom': [(0.0, 3.0), (2.6, 8.0)],
         'kitchen': [(0.0, 3.0), (2.6, 8.0)],
         'bathroom': [(6.1, 3.3), (8.0, 6.4)],
		 'storage': [(2.8, 6.7), (8.0, 8.0)]} 

In [ ]:
A = [(([4, 8], 'storage', 'motion sensors'), ([6, 4], 'bedroom', 'motion sensors'), ([7, 5], 'bathroom', 'motion sensors'), ([3, 8], 'storage', 'motion sensors'), ([4, 8], 'storage', 'motion sensors'), ([8, 6], 'bathroom', 'motion sensors'), ([8, 0], 'bedroom', 'motion sensors'), ([8, 5], 'bathroom', 'motion sensors'), ([8, 4], 'bedroom', 'motion sensors'), ([3, 3], 'livingroom', 'motion sensors'), ([8, 4], 'bedroom', 'motion sensors'), ([3, 3], 'livingroom', 'motion sensors'), ([5, 6], 'livingroom', 'motion sensors'), ([1, 8], 'diningroom', 'motion sensors'), ([1, 8], 'diningroom', 'motion sensors'))]

In [ ]:
A

In [ ]:
for a in A:
    for aa in a:
        print(aa)
        print('----')

In [ ]:
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

In [ ]:
df

In [ ]:
df